<a href="https://colab.research.google.com/github/SanGyuk-Raccoon/DACON_1/blob/main/ver_5_use_updated_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd   
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder, LabelEncoder

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

from lightgbm import LGBMClassifier
import xgboost

from sklearn.metrics import log_loss

In [58]:
# Load DATA
train = pd.read_csv("/content/drive/MyDrive/DACON/[월간 데이콘] 14. 신용카드 사용자 연체 예측 AI/train_update.csv")

test = pd.read_csv("/content/drive/MyDrive/DACON/[월간 데이콘] 14. 신용카드 사용자 연체 예측 AI/test_update.csv")


In [59]:
train.isnull().sum(axis = 0 )

index            0
gender           0
car              0
reality          0
child_num        0
income_total     0
income_type      0
edu_type         0
family_type      0
house_type       0
DAYS_BIRTH       0
DAYS_EMPLOYED    0
FLAG_MOBIL       0
work_phone       0
phone            0
email            0
occyp_type       0
family_size      0
begin_month      0
credit           0
dtype: int64

In [60]:
def continuous_to_quantile(var, k) :

  t = 1 / k
  q = 0
  Q = [np.quantile( data[var] , q)]

  for i in range(k) :
    q += t
    Q1 = np.quantile( data[var] , q)
    Q.append(Q1)

  for i in range(k-1) :
    data.loc[(data[var] >= Q[i]) & (data[var] <= Q[i+1]), f'{var}_Quantile_{k}'] = i

In [61]:
def pre_processing(DATA) :
  data =  DATA

  data = data.drop(['index', 'FLAG_MOBIL'], axis = 1)

  #### 1. gender
  mapping_gender = {
      'F' : 0, # Female = 0
      'M' : 1  # Male = 1
  }
  data['gender_mapping'] = data.gender.map(mapping_gender)


  # 2. car
  mapping_car = {
      'N' : 0,
      'Y' : 1
  }
  data['car_mapping'] = data.car.map(mapping_car)

  # 3. reality
  mapping_reality = {
      'N' : 0,
      'Y' : 1
  }
  data['reality_mapping'] = data.reality.map(mapping_reality)

  # 4. edu_type : Ordinal variable
  mapping_edu_type = {
    'Lower secondary' : 0, # 중학교 미만
    'Secondary / secondary special' : 1, #중학교
    'Incomplete higher' : 2, # 고등학교 중퇴
    'Higher education' : 3, # 고등학교 졸업
    'Academic degree' : 4 # 학사 이상
  }
  data['edu_type_ordinal_mapping'] = data.edu_type.map(mapping_edu_type)

  ## child num. 5를 최대로 설정
  data['child_num_max5'] = data['child_num'].apply( lambda x : 5 if x > 5 else x)

  ## family_size 7을 최대로 설정
  data['family_size_max7'] = data['family_size'].apply( lambda x : 7 if x > 7 else x)

  ## DAYS_BIRTH -> 나이
  data['age'] = np.floor(-data['DAYS_BIRTH']/ 365)
  data['age_year'] = -data['DAYS_BIRTH'] / 365 
  data['age_month'] = -data['DAYS_BIRTH'] / (365 / 12)
  data['age_month'] = -data['DAYS_BIRTH']

  continuous_to_quantile('age', 4)
  continuous_to_quantile('age', 5)


  ## DAYS_EMPLOYES -> 경력
  data['career_year'] = data['DAYS_EMPLOYED'].apply( lambda x : - x / 365  if x < 0 else 0)
  data['career_month'] = data['DAYS_EMPLOYED'].apply( lambda x : - x / (365 / 12) if x < 0 else 0)
  data['career_day'] = data['DAYS_EMPLOYED'].apply( lambda x : - x if x < 0 else 0)
  data['career'] = data['DAYS_EMPLOYED'].apply( lambda x : 1 if x < 0 else 0)



  ## Begin_month -> 양수로
  data['begin_month_minus'] = - data['begin_month']
  data['log_begin_month_minus'] = np.log(data['begin_month_minus'])


  #### one-hot encoding
  var = ['income_type',
        'family_type',
        'house_type',
        'occyp_type']



  onehot_encoder = OneHotEncoder()
  onehot_encoder.fit(data.loc[:, var])
  onehot_df = pd.DataFrame(onehot_encoder.transform(data.loc[:,var]).toarray(), 
              columns=onehot_encoder.get_feature_names(var))

  data = pd.concat([data, onehot_df], axis = 1)


  ######################################################
  data = data.drop(['gender', 'car', 'reality', 'edu_type'] + var, axis = 1)

  return data

In [62]:
#############카테고리 곱하기######################################
#for i in range(len(var)) :
#  for j in range(i+1, len(var)) :
#    data[f'{var[i]}_{var[j]}'] = data[var[i]] + '_' + data[var[j]]
#    var.append(f'{var[i]}_{var[j]}')
##################################################################
train = pre_processing(train)
train.head()

,child_num,income_total,DAYS_BIRTH,DAYS_EMPLOYED,work_phone,phone,email,family_size,begin_month,credit,gender_mapping,car_mapping,reality_mapping,edu_type_ordinal_mapping,child_num_max5,family_size_max7,age,age_year,age_month,career_year,career_month,career_day,career,begin_month_minus,log_begin_month_minus,income_type_Commercial associate,income_type_Pensioner,income_type_State servant,income_type_Student,income_type_Working,family_type_Civil marriage,family_type_Married,family_type_Separated,family_type_Single / not married,family_type_Widow,house_type_Co-op apartment,house_type_House / apartment,house_type_Municipal apartment,house_type_Office apartment,house_type_Rented apartment,house_type_With parents,occyp_type_Accountants,occyp_type_Cleaning staff,occyp_type_Cooking staff,occyp_type_Core staff,occyp_type_Drivers,occyp_type_HR staff,occyp_type_High skill tech staff,occyp_type_IT staff,occyp_type_Laborers,occyp_type_Low-skill Laborers,occyp_type_Managers,occyp_type_Medicine staff,occyp_type_NO_WORK,occyp_type_Private service staff,occyp_type_Realty agents,occyp_type_Sales staff,occyp_type_Secretaries,occyp_type_Security staff,occyp_type_Waiters/barmen staff
0,0,202500.0,-13899,-4709,0,0,0,2.0,-6.0,1.0,0,0,0,3,0,2.0,38.0,38.079452,13899,12.901370,154.816438,4709,1,6.0,1.791759,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,247500.0,-11380,-1540,0,0,1,3.0,-5.0,1.0,0,0,1,1,1,3.0,31.0,31.178082,11380,4.219178,50.630137,1540,1,5.0,1.609438,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,450000.0,-19087,-4434,0,1,0,2.0,-22.0,2.0,1,1,1,3,0,2.0,52.0,52.293151,19087,12.147945,145.775342,4434,1,22.0,3.091042,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,202500.0,-15088,-2092,0,1,0,2.0,-37.0,0.0,0,0,1,1,0,2.0,41.0,41.336986,15088,5.731507,68.778082,2092,1,37.0,3.610918,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0,157500.0,-15037,-2105,0,0,0,2.0,-26.0,2.0,0,1,1,3,0,2.0,41.0,41.197260,15037,5.767123,69.205479,2105,1,26.0,3.258097,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [63]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
folds=[]
for train_idx, valid_idx in skf.split(train, train['credit']):
    folds.append((train_idx, valid_idx))

lgb_models={}
lgb_logloss = []
for fold in range(5):
    print(f'===================================={fold+1}============================================')
    train_idx, valid_idx = folds[fold]
    X_train, X_valid, y_train, y_valid = train.drop(['credit'],axis=1).iloc[train_idx].values, train.drop(['credit'],axis=1).iloc[valid_idx].values,\
                                         train['credit'][train_idx].values, train['credit'][valid_idx].values 
    lgb = LGBMClassifier(n_estimators=1000)
    lgb.fit(X_train, y_train,
            eval_set=[(X_train, y_train), (X_valid, y_valid)], 
            early_stopping_rounds=30,
           verbose=100)
    lgb_logloss.append(lgb.best_score_['valid_1']['multi_logloss'])
    lgb_models[fold]=lgb
    print(f'================================================================================\n\n')

====================================1============================================
Training until validation scores don't improve for 30 rounds.
[100]	training's multi_logloss: 0.678816	valid_1's multi_logloss: 0.758263
[200]	training's multi_logloss: 0.603197	valid_1's multi_logloss: 0.739483
[300]	training's multi_logloss: 0.547643	valid_1's multi_logloss: 0.731671
[400]	training's multi_logloss: 0.501555	valid_1's multi_logloss: 0.726755
Early stopping, best iteration is:
[450]	training's multi_logloss: 0.481159	valid_1's multi_logloss: 0.725282


====================================2============================================
Training until validation scores don't improve for 30 rounds.
[100]	training's multi_logloss: 0.675392	valid_1's multi_logloss: 0.770406
[200]	training's multi_logloss: 0.59981	valid_1's multi_logloss: 0.752687
[300]	training's multi_logloss: 0.544478	valid_1's multi_logloss: 0.747944
[400]	training's multi_logloss: 0.499437	valid_1's multi_logloss: 0.745032
E

In [72]:
Submission = pd.read_csv("/content/drive/MyDrive/DACON/[월간 데이콘] 14. 신용카드 사용자 연체 예측 AI/sample_submission.csv")
Submission.iloc[:,1:]=0
for fold in range(5):
    Submission.iloc[:,1:] += lgb_models[fold].predict_proba(X_test)/5

Submission.to_csv("/content/drive/MyDrive/DACON/[월간 데이콘] 14. 신용카드 사용자 연체 예측 AI/use_update_data_LGB.csv", index = False)

In [64]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
folds=[]
for train_idx, valid_idx in skf.split(train, train['credit']):
    folds.append((train_idx, valid_idx))

params = {
    'max_depth' : 8,
    'eta' : 0.1,
    'objective' : 'multi:softprob',
    'eval_metric' : 'mlogloss',
    'early_stoppings' : 100,
    'num_class' : 3,
    'sub_sample' : 0.8
}

xgb_models={}
xgb_logloss = []

for fold in range(5):
    print(f'===================================={fold+1}============================================')
    train_idx, valid_idx = folds[fold]
    X_train, X_valid, y_train, y_valid = train.drop(['credit'],axis=1).iloc[train_idx].values, train.drop(['credit'],axis=1).iloc[valid_idx].values,\
                                         train['credit'][train_idx].values, train['credit'][valid_idx].values 
    dtrain = xgboost.DMatrix(data = X_train, label = y_train)
    dvalid = xgboost.DMatrix(data = X_valid, label = y_valid)                                     
    
    eval_list = [(dtrain, 'train'),
             (dvalid, 'valid')]

    xgb_model = xgboost.train(params = params,
                      dtrain = dtrain,
                      evals = eval_list,
                      num_boost_round = 400,
                      verbose_eval = 100
                      )
    

    xgb_models[fold]=xgb_model
    print(f'================================================================================\n\n')

====================================1============================================
[0]	train-mlogloss:1.05155	valid-mlogloss:1.05411
[100]	train-mlogloss:0.622446	valid-mlogloss:0.748642
[200]	train-mlogloss:0.525512	valid-mlogloss:0.731859
[300]	train-mlogloss:0.451792	valid-mlogloss:0.726749
[399]	train-mlogloss:0.394719	valid-mlogloss:0.728031


====================================2============================================
[0]	train-mlogloss:1.05137	valid-mlogloss:1.0546
[100]	train-mlogloss:0.607835	valid-mlogloss:0.756538
[200]	train-mlogloss:0.505851	valid-mlogloss:0.741266
[300]	train-mlogloss:0.435218	valid-mlogloss:0.74002
[399]	train-mlogloss:0.379385	valid-mlogloss:0.744579


====================================3============================================
[0]	train-mlogloss:1.05159	valid-mlogloss:1.05431
[100]	train-mlogloss:0.620213	valid-mlogloss:0.755092
[200]	train-mlogloss:0.514533	valid-mlogloss:0.738633
[300]	train-mlogloss:0.436076	valid-mlogloss:0.736754
[399]	tr

In [65]:
X_test = pre_processing(test)
dtest = xgboost.DMatrix(data = X_test)

In [67]:
Submission = pd.read_csv("/content/drive/MyDrive/DACON/[월간 데이콘] 14. 신용카드 사용자 연체 예측 AI/sample_submission.csv")
Submission.iloc[:,1:]=0
for fold in range(5):
    Submission.iloc[:,1:] += xgb_models[fold].predict(dtest, validate_features = False)/5

Submission.to_csv("/content/drive/MyDrive/DACON/[월간 데이콘] 14. 신용카드 사용자 연체 예측 AI/use_update_data_XGB.csv", index = False)